In [ ]:
# %matplotlib widget
import cv2
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import pickle
import datetime
from pathlib import Path
from IPython.display import display


from utils import (
    load_dataset,
    get_images,
    transform_points,
    get_densities,
    predict,
    calculate_errors,
    get_result_dict,
    SCALING_FACTOR,
    plot_setup, 
    plot_setup_noised,
    plot_predictions,
    add_outlier,
    get_numpy_arr
)

# Get Fov Points

In [ ]:
pv_img = 'IMG_01'
tv_img = 'IMG_00'
img_pv, img_tv = get_images(pv_img, tv_img)

In [ ]:
fov_pts_0026 = [
    [8.009250710227207, 3014.859952059659],
    [39.4274325284091, 1184.7508611505682],
    [1979.5001598011354, 666.3508611505681],
    [2270.1183416193176, 909.8417702414772],
    [2277.972887073863, 1231.8781338778408],
    [2513.609250710226, 1271.1508611505683],
    [3102.700159801135, 540.678133877841],
    [4029.536523437499, 898.0601917613635],
    [4021.6819779829534, 3010.932919034091]
]

fov_pts_0029 = [[]]
fov_pts_0032 = [[]]
fov_pts_0032 = [[]]

# Draw FoVs

In [ ]:
pv_img = 'IMG_01'
tv_img = 'IMG_00'
fov_pts_pv = fov_pts_0026
position = [2719.51263607, 1594.32214866, -551.05376955]

img_pv, img_tv = get_images(pv_img, tv_img)

In [ ]:
reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
reference_pts_pv_arr, reference_pts_tv_arr, validation_pts_pv_arr, validation_pts_tv_arr = transform_points(
        reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv)

print(reference_pts_pv_arr)
print(reference_pts_tv_arr)
# Calculate Homography
h, _ = cv2.findHomography(
    reference_pts_pv_arr,
    reference_pts_tv_arr,
    # method = cv2.RANSAC,
    method = 0,
)
h_inv = np.linalg.inv(h)

In [ ]:
fov_pts_pv_arr = [[pt[0], pt[1], 1] for pt in fov_pts_pv]
fov_pts_pv_arr = np.array(fov_pts_pv_arr)

In [ ]:
predicted_fov_pts = predict(fov_pts_pv_arr, h)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

img_tv = cv2.imread('data/images/IMG_00.JPG', cv2.IMREAD_GRAYSCALE)
img_pv = cv2.imread('data/images/IMG_01.JPG', cv2.IMREAD_GRAYSCALE)

axs[0].imshow(img_pv, cmap='gray', alpha=0.5)
#FF6347
# axs[0].fill(fov_pts_pv_arr[:, 0], fov_pts_pv_arr[:, 1], c=(1, 99/255, 71/255, 0.4), edgecolor='black')
# axs[0].fill(fov_pts_pv_arr[:, 0], fov_pts_pv_arr[:, 1], c=(70/255, 130/255, 180/255, 0.4), edgecolor='black')
axs[0].fill(fov_pts_pv_arr[:, 0], fov_pts_pv_arr[:, 1], c=(0.267004, 0.004874, 0.329415, 0.4), edgecolor='black')
axs[0].scatter(fov_pts_pv_arr[:, 0], fov_pts_pv_arr[:, 1], marker='o', edgecolors='black', color=(0.267004, 0.004874, 0.329415, 1.0), s=50)
# ax.scatter(position[0], position[1], marker='D', edgecolors='black', color='tomato', s=200)
axs[0].axis('off')

axs[1].imshow(img_tv, cmap='gray', alpha=0.5)
# axs[1].fill(predicted_fov_pts[:, 0], predicted_fov_pts[:, 1], c=(70/255, 130/255, 180/255, 0.4), edgecolor='black')
axs[1].fill(predicted_fov_pts[:, 0], predicted_fov_pts[:, 1], c=(0.267004, 0.004874, 0.329415, 0.4), edgecolor='black')
# axs[1].fill(predicted_fov_pts[:, 0], predicted_fov_pts[:, 1], c=(1, 99/255, 71/255, 0.4), edgecolor='black')
axs[1].scatter(predicted_fov_pts[:, 0], predicted_fov_pts[:, 1], marker='o', edgecolors='black', color=(0.267004, 0.004874, 0.329415), s=50)

axs[1].scatter(position[0], position[1], marker='H', edgecolors='black', color=(0.267004, 0.004874, 0.329415), s=300)
axs[1].axis('off')


plt.tight_layout()
plt.savefig('output/predicted_fov.png', dpi=100)
plt.show()